In [1]:
import polars as pl

df = pl.DataFrame({"x": [[1, 2, 3], [6, 5, 4]]})

print(
    df.with_columns(
        y = pl.col.x + 3,
        x_max = pl.col("x").list.max(),
        x_sum = pl.col("x").list.sum()
    ).with_columns(
        z = pl.col.x * pl.col.y,
        z2 = pl.col("x").list.eval((pl.element() >= 2) & (pl.element() <= 4))
    )
)

# print( pl.col("x").list )
# print( pl.select(pl.col("x").list) )





print("\n=================================================\n")


## only when the column starts with x and string starts with foo, then uppercase all string columns.

df = pl.DataFrame(
    {
        "x": ["foo1", "bar1", "foo2", "foo3", "foo4"],
        "y": [6, 4, 5, 1, 2],
        "z1": ["abc", "def", "ghi", "foo1", "foooooo11"],
        "z2": ["jkl", "mno", "prq", "foo3", "foooooo22"]
    }
)

print(
    df.with_columns(
        pl.when(pl.col("x").str.starts_with("foo"))
          .then(pl.col(pl.String).str.to_uppercase())
          .otherwise(pl.col(pl.String))
    )
)



shape: (2, 6)
┌───────────┬───────────┬───────┬───────┬──────────────┬──────────────────────┐
│ x         ┆ y         ┆ x_max ┆ x_sum ┆ z            ┆ z2                   │
│ ---       ┆ ---       ┆ ---   ┆ ---   ┆ ---          ┆ ---                  │
│ list[i64] ┆ list[i64] ┆ i64   ┆ i64   ┆ list[i64]    ┆ list[bool]           │
╞═══════════╪═══════════╪═══════╪═══════╪══════════════╪══════════════════════╡
│ [1, 2, 3] ┆ [4, 5, 6] ┆ 3     ┆ 6     ┆ [4, 10, 18]  ┆ [false, true, true]  │
│ [6, 5, 4] ┆ [9, 8, 7] ┆ 6     ┆ 15    ┆ [54, 40, 28] ┆ [false, false, true] │
└───────────┴───────────┴───────┴───────┴──────────────┴──────────────────────┘


shape: (5, 4)
┌──────┬─────┬───────────┬───────────┐
│ x    ┆ y   ┆ z1        ┆ z2        │
│ ---  ┆ --- ┆ ---       ┆ ---       │
│ str  ┆ i64 ┆ str       ┆ str       │
╞══════╪═════╪═══════════╪═══════════╡
│ FOO1 ┆ 6   ┆ ABC       ┆ JKL       │
│ bar1 ┆ 4   ┆ def       ┆ mno       │
│ FOO2 ┆ 5   ┆ GHI       ┆ PRQ       │
│ FOO3 ┆ 1   ┆ FOO

In [7]:
import polars as pl

# Create the data
toy = pl.DataFrame({
    "docID": ["A_2020","A_2020","B_2021"],
    "tickers": [["ABC","ABC.A"], ["ABC","ABC.A"], ["XYZ"]],
    "exchanges": [["NYSE"], ["NYSE"], ["NASDAQ"]],
    "alt": [["ALT1"], ["ALT2"], ["ALT3"]],
})

print("Original DataFrame:")
print(toy)
print("\n" + "="*50 + "\n")

# ============================================
# STEP BY STEP: Understanding the data structure
# ============================================

# First, let's see what we're working with
print("For docID 'A_2020', we have these ticker lists:")
a_2020_rows = toy.filter(pl.col("docID") == "A_2020")
print(a_2020_rows.select("tickers"))
print("We want to merge: [['ABC','ABC.A'], ['ABC','ABC.A']] into one list")
print("\n" + "="*50 + "\n")

# ============================================
# METHOD 1: EXPLODE + IMPLODE (Most reliable)
# ============================================
print("METHOD 1: Explode then Implode")

# Step 1: Explode lists into individual rows
exploded = toy.select(["docID", "tickers"]).explode("tickers")
print("After exploding lists:")
print(exploded)
print()

# Step 2: Group and collect back into lists
result1 = (
    exploded.group_by("docID")
    .agg(
        pl.col("tickers").unique().sort().alias("merged_tickers")
    )
)
print("After grouping and collecting:")
print(result1)
print("\n" + "="*50 + "\n")

# ============================================
# METHOD 2: Using FLATTEN in aggregation (simplest)
# ============================================
print("METHOD 2: Direct flatten in aggregation")

result2 = (
    toy.group_by("docID")
    .agg(
        # Just flatten - Polars will auto-collect into a list
        pl.col("tickers").flatten().unique().sort().alias("merged_tickers")
    )
)
print(result2)
print("\n" + "="*50 + "\n")


Original DataFrame:
shape: (3, 4)
┌────────┬──────────────────┬────────────┬───────────┐
│ docID  ┆ tickers          ┆ exchanges  ┆ alt       │
│ ---    ┆ ---              ┆ ---        ┆ ---       │
│ str    ┆ list[str]        ┆ list[str]  ┆ list[str] │
╞════════╪══════════════════╪════════════╪═══════════╡
│ A_2020 ┆ ["ABC", "ABC.A"] ┆ ["NYSE"]   ┆ ["ALT1"]  │
│ A_2020 ┆ ["ABC", "ABC.A"] ┆ ["NYSE"]   ┆ ["ALT2"]  │
│ B_2021 ┆ ["XYZ"]          ┆ ["NASDAQ"] ┆ ["ALT3"]  │
└────────┴──────────────────┴────────────┴───────────┘


For docID 'A_2020', we have these ticker lists:
shape: (2, 1)
┌──────────────────┐
│ tickers          │
│ ---              │
│ list[str]        │
╞══════════════════╡
│ ["ABC", "ABC.A"] │
│ ["ABC", "ABC.A"] │
└──────────────────┘
We want to merge: [['ABC','ABC.A'], ['ABC','ABC.A']] into one list


METHOD 1: Explode then Implode
After exploding lists:
shape: (5, 2)
┌────────┬─────────┐
│ docID  ┆ tickers │
│ ---    ┆ ---     │
│ str    ┆ str     │
╞════════╪════════

In [9]:
import polars as pl

# Create the data
toy = pl.DataFrame({
    "docID": ["A_2020","A_2020","B_2021"],
    "tickers": [["ABC","ABC.A"], ["ABC","ABC.A"], ["XYZ"]],
    "exchanges": [["NYSE"], ["NYSE"], ["NASDAQ"]],
    "alt": [["ALT1"], ["ALT2"], ["ALT3"]],
})

print("Original DataFrame:")
print(toy)
print("\n" + "="*50 + "\n")

# ============================================
# THE SIMPLEST WORKING APPROACH
# ============================================

# APPROACH 1: Just use flatten() in aggregation - IT WORKS!
print("SIMPLEST APPROACH - Just flatten in agg:")
result = (
    toy.group_by("docID")
    .agg(
        pl.col("tickers").flatten()  # That's it!
    )
)
print(result)
print()

# Want unique values? Add .unique()
result_unique = (
    toy.group_by("docID")
    .agg(
        pl.col("tickers").flatten().unique()
    )
)
print("With unique:")
print(result_unique)
print("\n" + "="*50 + "\n")

# ============================================
# UNDERSTANDING WHAT FLATTEN DOES
# ============================================

print("Let's see what happens step by step:")

# For docID A_2020:
# Row 1: ["ABC", "ABC.A"]
# Row 2: ["ABC", "ABC.A"]
# After flatten in group: ["ABC", "ABC.A", "ABC", "ABC.A"]
# After unique: ["ABC", "ABC.A"]

# Let's verify this manually
a_2020_only = toy.filter(pl.col("docID") == "A_2020")
print("Rows for A_2020:")
print(a_2020_only.select("tickers"))
print()

# ============================================
# CONCAT LISTS FROM DIFFERENT COLUMNS
# ============================================

print("Row-wise concatenation (across columns):")

# This combines tickers, exchanges, alt FROM THE SAME ROW
row_concat = toy.select([
    pl.concat_list(["tickers", "exchanges", "alt"]).alias("combined")
])
print(row_concat)
# Row 0: ["ABC", "ABC.A"] + ["NYSE"] + ["ALT1"] = ["ABC", "ABC.A", "NYSE", "ALT1"]
print("\n" + "="*50 + "\n")

# ============================================
# COMBINE BOTH: Concat columns, then merge rows
# ============================================

print("First concat columns, then merge rows per group:")

# Step 1: Concat columns
with_concat = toy.with_columns(
    pl.concat_list(["tickers", "exchanges"]).alias("ticker_and_exchange")
)
print("After concatenating columns:")
print(with_concat.select(["docID", "ticker_and_exchange"]))
print()

# Step 2: Group and flatten
final = (
    with_concat.group_by("docID")
    .agg(
        pl.col("ticker_and_exchange").flatten().unique().sort()
    )
)
print("After grouping and flattening:")
print(final)
print("\n" + "="*50 + "\n")

# ============================================
# IF YOU NEED MORE CONTROL - Use explode/implode
# ============================================

print("Alternative: Explode then re-aggregate")

# This physically creates new rows
exploded = toy.explode("tickers")
print("After exploding (creates new rows):")
print(exploded)
print()

# Then group back
regrouped = (
    exploded.group_by("docID")
    .agg(
        pl.col("tickers").unique().sort()  # Automatically collects into list
    )
)
print("After regrouping:")
print(regrouped)


simplertech = toy.group_by("docID").agg(
    pl.col("tickers").flatten().unique().sort()
)
print("\nSimplertech result:", simplertech)

Original DataFrame:
shape: (3, 4)
┌────────┬──────────────────┬────────────┬───────────┐
│ docID  ┆ tickers          ┆ exchanges  ┆ alt       │
│ ---    ┆ ---              ┆ ---        ┆ ---       │
│ str    ┆ list[str]        ┆ list[str]  ┆ list[str] │
╞════════╪══════════════════╪════════════╪═══════════╡
│ A_2020 ┆ ["ABC", "ABC.A"] ┆ ["NYSE"]   ┆ ["ALT1"]  │
│ A_2020 ┆ ["ABC", "ABC.A"] ┆ ["NYSE"]   ┆ ["ALT2"]  │
│ B_2021 ┆ ["XYZ"]          ┆ ["NASDAQ"] ┆ ["ALT3"]  │
└────────┴──────────────────┴────────────┴───────────┘


SIMPLEST APPROACH - Just flatten in agg:
shape: (2, 2)
┌────────┬─────────────────────────────┐
│ docID  ┆ tickers                     │
│ ---    ┆ ---                         │
│ str    ┆ list[str]                   │
╞════════╪═════════════════════════════╡
│ B_2021 ┆ ["XYZ"]                     │
│ A_2020 ┆ ["ABC", "ABC.A", … "ABC.A"] │
└────────┴─────────────────────────────┘

With unique:
shape: (2, 2)
┌────────┬──────────────────┐
│ docID  ┆ tickers          

In [14]:
import polars as pl

# ============================================================================
# EXAMPLE 1: SIMPLEST WINDOW FUNCTION - Understanding .over()
# ============================================================================

# Minimal dataset: sales data
df = pl.DataFrame({
    "store": ["A", "A", "A", "B", "B", "B"],
    "day":   [1, 2, 3, 1, 2, 3],
    "sales": [100, 150, 200, 80, 90, 110]
})

print("Original Data:")
print(df)
print()

# Add store total using window function
df_with_total = df.with_columns(
    store_total = pl.col("sales").sum().over("store")
)

print("With Store Totals (window function):")
print(df_with_total)
print()

# What happened?
# - .sum() alone would give ONE value: 630
# - .sum().over("store") gives sum PER STORE, repeated for each row
# - Store A: 100+150+200 = 450 (appears 3 times)
# - Store B: 80+90+110 = 280 (appears 3 times)

# ============================================================================
# EXAMPLE 2: WINDOW vs GROUP_BY - The Key Difference
# ============================================================================

# GROUP_BY: Collapses rows (1 row per group)
grouped = df.group_by("store").agg(
    pl.col("sales").sum().alias("total_sales")
)
print("GROUP_BY Result (collapsed):")
print(grouped)  # Only 2 rows!
print()

# WINDOW: Keeps all rows, adds calculation

windowed = df.with_columns([
    pl.col("sales").sum().over("store").alias("total"),
    (pl.col("sales") / pl.col("sales").sum().over("store") * 100).round(1).alias("pct_of_store")
])
print("WINDOW Result (all rows preserved):")
print(windowed)  # Still 6 rows!
print()

# ============================================================================
# EXAMPLE 3: ROW NUMBER and RANKING
# ============================================================================

# Most common window function: numbering rows within groups
df_ranked = df.with_columns([
    # Row number within each store (1, 2, 3...)
    (pl.int_range(0, pl.len()).over("store") + 1).alias("row_num"),
    
    # Rank by sales within each store
    pl.col("sales").rank().over("store").alias("rank"),
    
    # Is this the max sale for this store?
    (pl.col("sales") == pl.col("sales").max().over("store")).alias("is_best_day")
])

print("Rankings within groups:")
print(df_ranked)
print()

# ============================================================================
# EXAMPLE 4: MULTIPLE COLUMNS IN .over() - Finer Partitions
# ============================================================================

# More complex data with regions
df2 = pl.DataFrame({
    "region": ["East", "East", "East", "East", "West", "West", "West", "West"],
    "store":  ["A", "A", "B", "B", "A", "A", "B", "B"],
    "month":  ["Jan", "Feb", "Jan", "Feb", "Jan", "Feb", "Jan", "Feb"],
    "sales":  [100, 120, 80, 90, 150, 160, 110, 130]
})

print("Multi-level data:")
print(df2)
print()

# Different window partitions
df2_windows = df2.with_columns([
    pl.col("sales").sum().over("region").alias("region_total"),
    pl.col("sales").sum().over("store").alias("store_total"),
    pl.col("sales").sum().over(["region", "store"]).alias("region_store_total"),
    (pl.int_range(0, pl.len()).over(["region", "store"]) + 1).alias("row_in_group")
])
print("Multiple window partitions:")
print(df2_windows)
print()

# ============================================================================
# EXAMPLE 5: PRACTICAL - Running Calculations
# ============================================================================

# Time series data
df3 = pl.DataFrame({
    "date": ["2024-01", "2024-02", "2024-03", "2024-04", "2024-05"],
    "product": ["A", "A", "A", "A", "A"],
    "units": [10, 15, 12, 18, 20]
})

df3_running = df3.with_columns([
    pl.col("units").cum_sum().over("product").alias("cumsum"),
    (pl.col("units").cum_sum().over("product") / 
     (pl.int_range(0, pl.len()).over("product") + 1)).alias("running_avg"),
    pl.col("units").shift(1).over("product").alias("prev_units"),
    (pl.col("units") - pl.col("units").shift(1).over("product")).alias("mom_change")
])


print("Running calculations:")
print(df3_running)
print()

# ============================================================================
# EXAMPLE 6: YOUR ACTUAL USE CASE - Document Position
# ============================================================================

# Simulating your SEC filings structure
docs = pl.DataFrame({
    "docID": ["DOC1", "DOC1", "DOC1", "DOC2", "DOC2", "DOC2"],
    "section": [1, 1, 2, 1, 2, 2],
    "sentence": ["First.", "Second.", "Third.", "Alpha.", "Beta.", "Gamma."],
    "sentenceCount": [1, 2, 3, 1, 2, 3]
})

print("Document sentences:")
print(docs)
print()

# Your exact pattern: position within document and section
docs_with_positions = (
    docs
    .sort(["docID", "sentenceCount"])
    .with_columns([
        pl.int_range(0, pl.len()).over("docID").alias("pos_in_doc"),
        pl.int_range(0, pl.len()).over(["docID", "section"]).alias("pos_in_sect"),
        pl.len().over("docID").alias("doc_total"),
        pl.len().over(["docID", "section"]).alias("sect_total")
    ])
)

print("With positions (like your code):")
print(docs_with_positions)
print()

# ============================================================================
# KEY CONCEPTS SUMMARY
# ============================================================================

print("="*60)
print("WINDOW FUNCTION MENTAL MODEL:")
print("="*60)
print("""
1. GROUP_BY vs WINDOW:
   - group_by(): COLLAPSES rows → 1 row per group
   - .over(): PRESERVES rows → adds column with group calculation

2. .over("column"):
   - Partitions data by that column
   - Calculates within each partition
   - Returns result for EVERY original row

3. .over(["col1", "col2"]):
   - Partitions by COMBINATION of columns
   - Like GROUP BY col1, col2 in SQL
   - Finer granularity than single column

4. Common Window Operations:
   - pl.int_range(0, pl.len()).over(group) → row number
   - pl.col(x).sum().over(group) → group total
   - pl.col(x).max().over(group) → group maximum
   - pl.col(x).shift(1).over(group) → previous value
   - pl.col(x).cum_sum().over(group) → running total

5. Your Use Case:
   - .over("docID") → calculations within each document
   - .over(["docID", "section"]) → within doc-section pairs
   - Perfect for: "position of sentence in document"
""")

# ============================================================================
# VISUAL REPRESENTATION
# ============================================================================

print("="*60)
print("VISUAL: How .over() Works")
print("="*60)

demo = pl.DataFrame({
    "group": ["A", "A", "B", "B"],
    "value": [10, 20, 5, 15]
})

print("Original:")
print(demo)
print()

print("With .sum().over('group'):")
result = demo.with_columns(
    pl.col("value").sum().over("group").alias("group_sum")
)

print(result)
print()

print("""
What happened internally:
1. Split by 'group': [A: [10,20], B: [5,15]]
2. Calculate sum per group: [A: 30, B: 20]
3. Broadcast back to original rows:
   - Row 0 (group A): gets 30
   - Row 1 (group A): gets 30
   - Row 2 (group B): gets 20
   - Row 3 (group B): gets 20
""")

Original Data:
shape: (6, 3)
┌───────┬─────┬───────┐
│ store ┆ day ┆ sales │
│ ---   ┆ --- ┆ ---   │
│ str   ┆ i64 ┆ i64   │
╞═══════╪═════╪═══════╡
│ A     ┆ 1   ┆ 100   │
│ A     ┆ 2   ┆ 150   │
│ A     ┆ 3   ┆ 200   │
│ B     ┆ 1   ┆ 80    │
│ B     ┆ 2   ┆ 90    │
│ B     ┆ 3   ┆ 110   │
└───────┴─────┴───────┘

With Store Totals (window function):
shape: (6, 4)
┌───────┬─────┬───────┬─────────────┐
│ store ┆ day ┆ sales ┆ store_total │
│ ---   ┆ --- ┆ ---   ┆ ---         │
│ str   ┆ i64 ┆ i64   ┆ i64         │
╞═══════╪═════╪═══════╪═════════════╡
│ A     ┆ 1   ┆ 100   ┆ 450         │
│ A     ┆ 2   ┆ 150   ┆ 450         │
│ A     ┆ 3   ┆ 200   ┆ 450         │
│ B     ┆ 1   ┆ 80    ┆ 280         │
│ B     ┆ 2   ┆ 90    ┆ 280         │
│ B     ┆ 3   ┆ 110   ┆ 280         │
└───────┴─────┴───────┴─────────────┘

GROUP_BY Result (collapsed):
shape: (2, 2)
┌───────┬─────────────┐
│ store ┆ total_sales │
│ ---   ┆ ---         │
│ str   ┆ i64         │
╞═══════╪═════════════╡
│ B     ┆ 2